In [1]:
#Based on: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [2]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import os
import time

/Applications/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
import numpy as np

In [118]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import re
import string
import sklearn
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn import tree
%matplotlib inline
import seaborn as sns
from sklearn.externals.six import StringIO
from IPython.display import Image
import pydotplus

In [88]:
curr_dir = os.getcwd()

In [89]:
df = pd.read_csv(curr_dir+'/data/ISIC-2017_Training_Data_metadata.csv')
df2 = pd.read_csv(curr_dir+'/data/ISIC-2017_Training_Part3_GroundTruth.csv')

In [132]:
 df = df.merge(df2, left_on='image_id', right_on='image_id', how='outer')
print('Number of samples: ', df.shape)

Number of samples:  (2000, 8)


In [131]:
df.head(5)

,Unnamed: 0,image_id,age_approximate,sex,melanoma,seborrheic_keratosis
0,0,ISIC_0000000,55,0,0.0,0.0
1,1,ISIC_0000001,30,0,0.0,0.0
2,2,ISIC_0000002,60,0,1.0,0.0
3,3,ISIC_0000003,30,1,0.0,0.0
4,4,ISIC_0000004,80,1,1.0,0.0


In [92]:
dict_df = {'age_max_min':[max([x  for x in df.age_approximate if x!='unknown']),min([x  for x in df.age_approximate if x!='unknown'])],
           'age_groups':set(df.age_approximate),
           'gender':set(df.sex),
          }

In [93]:
#dict_df.get('max_min_age')[0]=filter(lambda x: max(x), df.age_approximate)
set(df.age_approximate)
for x in df.age_approximate:
    if not x:
        print(x)

In [94]:
print('Number of unknown age: ', sum([1  for x in df.age_approximate if x=='unknown']))

Number of unknown age:  256


In [95]:
print('Number of unknown sex: ', sum([1  for x in df.sex if x=='unknown']))

Number of unknown sex:  229


In [96]:
dict_df['age_max_min']

['85', '10']

In [97]:
dict_df['age_groups']

{'10',
 '15',
 '20',
 '25',
 '30',
 '35',
 '40',
 '45',
 '5',
 '50',
 '55',
 '60',
 '65',
 '70',
 '75',
 '80',
 '85',
 'unknown'}

In [98]:
dict_df['gender']

{'female', 'male', 'unknown'}

In [99]:
df.columns

Index(['image_id', 'age_approximate', 'sex', 'melanoma',
       'seborrheic_keratosis'],
      dtype='object')

In [100]:
df.to_csv(curr_dir+'/data/combined.csv')

In [101]:
df = pd.read_csv(curr_dir+'/data/combined.csv')
df

,Unnamed: 0,image_id,age_approximate,sex,melanoma,seborrheic_keratosis
0,0,ISIC_0000000,55,female,0.0,0.0
1,1,ISIC_0000001,30,female,0.0,0.0
2,2,ISIC_0000002,60,female,1.0,0.0
3,3,ISIC_0000003,30,male,0.0,0.0
4,4,ISIC_0000004,80,male,1.0,0.0
5,5,ISIC_0000006,25,female,0.0,0.0
6,6,ISIC_0000007,25,female,0.0,0.0
7,7,ISIC_0000008,30,female,0.0,0.0
8,8,ISIC_0000009,30,female,0.0,0.0
9,9,ISIC_0000010,35,female,0.0,0.0


In [102]:
#df.sex= df.sex.map({'female': 0, 'male': 1,'unknown':2})
df.loc[df.age_approximate == 'unknown', 'age_approximate'] = 0
df.loc[df.sex == 'female', 'sex'] = 0
df.loc[df.sex == 'male', 'sex'] = 1
df.loc[df.sex == 'unknown', 'sex'] = 2

df

,Unnamed: 0,image_id,age_approximate,sex,melanoma,seborrheic_keratosis
0,0,ISIC_0000000,55,0,0.0,0.0
1,1,ISIC_0000001,30,0,0.0,0.0
2,2,ISIC_0000002,60,0,1.0,0.0
3,3,ISIC_0000003,30,1,0.0,0.0
4,4,ISIC_0000004,80,1,1.0,0.0
5,5,ISIC_0000006,25,0,0.0,0.0
6,6,ISIC_0000007,25,0,0.0,0.0
7,7,ISIC_0000008,30,0,0.0,0.0
8,8,ISIC_0000009,30,0,0.0,0.0
9,9,ISIC_0000010,35,0,0.0,0.0


In [103]:
df.to_csv(curr_dir+'/data/combined.csv')

In [133]:
df.shape

(2000, 8)

----

### Random Forest

In [105]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [106]:
columns=['age_approximate', 'sex']
X= df[columns]  ##this makes a copy
y=df[['melanoma']]
X.head(2)

,age_approximate,sex
0,55,0
1,30,0


In [107]:
#X.sex = X.sex.map({'female': 0, 'male': 1,'unknown':2}) #need to change all the values for this

In [108]:
#X.age_approximate = X.age_approximate.map({'unknown': '0'})
#X.loc[X.age_approximate == 'unknown', 'age_approximate'] = 0

In [109]:
for v in df.melanoma:
    print(v)

0.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
0.0
0.0
1.0
0.0
0.0
1.0
1.0
1.0
0.0
0.0
1.0
1.0
0.0
0.0
0.0
1.0
0.0
0.0
1.0
0.0
0.0
1.0
0.0
0.0
1.0
0.0
0.0
0.0
0.0
1.0
0.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
0.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0


In [110]:
dict_df

{'age_groups': {'10',
  '15',
  '20',
  '25',
  '30',
  '35',
  '40',
  '45',
  '5',
  '50',
  '55',
  '60',
  '65',
  '70',
  '75',
  '80',
  '85',
  'unknown'},
 'age_max_min': ['85', '10'],
 'gender': {'female', 'male', 'unknown'}}

In [111]:
X, y = make_classification(n_samples=1000, n_features=2,
                           n_informative=2, n_redundant=0,
                           random_state=0, shuffle=False)
clf = RandomForestClassifier(n_estimators=100, max_depth=10,
                             random_state=0)
#clf.fit(X, y)
#clf = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
#            max_depth=25, max_features='auto', max_leaf_nodes=None,
#            min_impurity_decrease=0.0, min_impurity_split=None,
#            min_samples_leaf=1, min_samples_split=2,
#            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
#            oob_score=False, random_state=0, verbose=0, warm_start=False)
clf.fit(X,y)
print(clf.feature_importances_)
#print(clf.predict([[85, 4]]))
print(clf.predict([[85, 1],[5, 0],[15, 0], [85, 2], [65, 1], [25, 0], [45, 1]]))

[0.11121574 0.88878426]
[1 0 0 1 1 0 1]


In [129]:
clf2 = tree.DecisionTreeClassifier(criterion='entropy', max_depth=3)
clf2=clf2.fit(X,y)
dot=StringIO()
tree.export_graphviz(clf2, out_file=dot, #feature_names = columns, class_names = ['0','1'], 
                                 filled = True, rounded = True,
                                 special_characters=True) 
graph = pydotplus.graph_from_dot_data(dot.getvalue())
#Image(graph.create_png())

p=graph.create_png
#plt.imread(p)
plt.show()